In [1]:
!pip -q install 'crewai[tools]'==0.28.8 --progress-bar off
!pip -q install duckduckgo-search==5.3.0 --progress-bar off

In [2]:
import json
import os
import re
import requests
from datetime import datetime
from crewai import Agent, Crew, Process, Task
from google.colab import userdata
from IPython.display import Markdown
from langchain.agents import load_tools
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI

In [3]:
def format_response(response: str) -> str:
    entries = re.split(r"(?<=]), (?=\[)", response)
    return [entry.strip("[]") for entry in entries]

In [4]:
os.environ["OPENAI_API_KEY"] = "" # Input your openai api key

In [5]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=5)

In [6]:
response = search_tool.run("Bitcoin")
format_response(response)

['snippet: Still, bitcoin may resume its rally in September, Lee said. The cryptocurrency is likely to be boosted by potential rate cuts from the Federal Reserve then, he said. Traders are now pricing in two rate cuts by the end of the year, starting in September, according to the CME FedWatch Tool., title: Bitcoin is trading sideways, but get ready for another rally in September, link: https://www.msn.com/en-us/money/other/bitcoin-is-trading-sideways-but-get-ready-for-another-rally-in-september/ar-BB1m3CJN, date: 2024-05-08T20:27:00+00:00, source: MarketWatch on MSN.com',
 "snippet: A long-term moving bitcoin price average hit a new high, diminishing concerns about volatility and pointing to future growth., title: New bitcoin record signals 'parabolic phase' for price, analyst says, link: https://www.thestreet.com/crypto/markets/new-bitcoin-average-price-record-signals-parabolic-phase-for-crypto-market, date: 2024-05-08T16:18:00+00:00, source: TheStreet.com",
 'snippet: Pan said on Li

In [7]:
human_tools = load_tools(["human"])

In [8]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.5
)

In [9]:
customer_communicator = Agent(
    role="Senior cryptocurrency customer communicator",
    goal="Find which cryptocurrencies the customer is interested in",
    backstory="""You're highly experienced in communicating about cryptocurrencies and blockchain technology with customers and their research needs""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=human_tools
)

In [10]:
analyst = Agent(
    role="Cryptocurrency Analyst",
    goal="""Create a short summary of the market news. Specify the current trend - up，down or sideways. For each requested cryptocurrency asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed.""",
    backstory="""You're widely accepted as the best cryptocurrency analyst that understands the market and have tracked every asset for more than 10 years. Your trendsanalysis are always extremely accurate.
    You're also master level analyst in the traditional markets and have deep understanding of human psychology, You understand macro factors and combine those multiple theories - e.g. cycle theory. You're able to hold multiple opininons when analysing anything.
    You understand news, their titles and information, but you look at those with a healthy dose of skepticism. You also consider the source of news articles.
    Your most well developed talent is providing clear and concise summarization that explains very complex market topics in simple to understand terms.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[search_tool]
)

In [11]:
writer = Agent(
    role="Cryptocurrency Newsletter Writer",
    goal="Write an insightful, compelling and informative 3 paragraph long newsletter based on the cryptocurrency report.",
    backstory="""You're deeply familiar with all cryptocurrency assets and blockchain technology. You understand complex concepts and create compelling stories and narratives that resonate with wider audiences.
    You're a master of communicating effectively and create narratives that engage the audience.
    Your writing focuses on conciseness, clearness and easy to read (flow) formatting.
    Some of your writing techniques include:
    - Start with a 3 bullet list (executive summary) of the most importannt points
    - Use bullet points and subheadings to make text easier to understand and read
    - Use analogies to convey complex ideas
    - Write compelling introductions and conclusions that make your work memorable
    You writing transforms even dry and most technical texts into a pleasant and interesting read.
    """,
    llm=llm,
    verbose=True,
    max_iter=5,
    memory=True,
    allow_delegation=True
)

In [12]:
get_cryptocurrencies_list = Task(
    description="ask which cryptocurrencies the customer is interested in.",
    expected_output = """Create a list of the cryptocurrencies that the human wants you to research.\n\n eg. cryptocurrencies = 'BTC，ETH'""",
    agent=customer_communicator
)

In [13]:
get_news = Task(
    description=f"""Take the list of cryptocurrencies and always include BTC and ETH to it (if not requested). Use the search tool to search for each asset individually.
    The current date is {datetime.now()}.
    Compose the results into a helpful report""",
    expected_output="""A summary of the overall cryptocurrency market and one sentence summary for each customer requested asset, Include a fear/greed score for each asset based on the news. Use the format:
    <CRYPTO ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
    """,
    agent=analyst,
    context=[get_cryptocurrencies_list]
)

In [14]:
write_newsletter = Task(
    description="""Use the cryptocurrency report to create a newsletter that is brief and highlights the most important points.
    Focus on the news, trend and fear/greed score, what are the near and future considerations?
    The newsletter is written daily and should be accessible to a wide audience.
    Don't rely on hype, but be clear and factual.
    """,
    expected_output="""An eloquent 3 paragraphs nwsletter formated as markdown in an easy readable manner. It should contain:
    - 3 bullets executive summary
    - introduction - set up the overall picture and spike up the interest (hook)
    - main part - provides the meat of the analysis including news summary and fear/greed scores
    - summary - key facts and a concrete future trend prediction - up, down or sideways
    """,
    agent=writer,
    context=[get_news]
)

In [15]:
crew = Crew(
    agents=[customer_communicator, analyst, writer],
    tasks=[get_cryptocurrencies_list, get_news, write_newsletter],
    verbose=2,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=5
)

In [16]:
results = crew.kickoff()

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: ask which cryptocurrencies the customer is interested in.


> Entering new CrewAgentExecutor chain...
I need to gather information on which cryptocurrencies the customer is interested in so that I can manage the team to research them effectively.

Action: Ask question to co-worker
Action Input: {"coworker": "Senior cryptocurrency customer communicator", "question": "Which cryptocurrencies are you interested in?", "context": "We need to know which cryptocurrencies the customer wants us to research."}

> Entering new CrewAgentExecutor chain...
I should start by asking the customer directly which cryptocurrencies they are interested in.

Action: human
Action Input: {"question": "Which cryptocurrencies are you interested in?"}

Which cryptocurrencies are you interested in?
Bitcoin
 

Bitcoin

Final Answer: The customer is interested in Bitcoin.

> Finished chain.
 

The customer is interested in Bitcoin.

Final Answer: cryptocur

In [17]:
list(results.keys())

['final_output', 'tasks_outputs']

In [18]:
len(results["tasks_outputs"])

3

In [19]:
Markdown(results["tasks_outputs"][1].exported_output)

Ethereum
The current sentiment in the Ethereum market is bearish due to factors such as the withdrawal of Grayscale's Ethereum futures ETF application and a correction near key Fibonacci support levels. Traders are starting to bet against ETH's price, and uncertainty surrounds whether it will break bearishly or potentially reverse. Proceed with caution when considering Ethereum investments at this time. Fear/Greed Score: 20

In [20]:
Markdown(results["final_output"])

# Ethereum Market Update - Daily Newsletter

## Executive Summary:
- Ethereum (ETH) is currently experiencing a bearish sentiment in the market.
- Traders are starting to bet against ETH's price due to key Fibonacci support levels.
- The Fear/Greed Score for Ethereum stands at 20, indicating caution is advised.

## Introduction:
The Ethereum market is currently facing a bearish sentiment as traders are becoming increasingly cautious. Recent events, such as the withdrawal of Grayscale's Ethereum futures ETF application and a correction near key Fibonacci support levels, have contributed to this negative outlook. It is essential to closely monitor the market conditions and exercise caution when considering investments in Ethereum at this time.

## Main Analysis:
Ethereum is correcting near key Fibonacci support levels, with uncertainty surrounding whether it will break bearishly or potentially reverse. The RSI indicator for Ethereum is currently neutral at 51, indicating an evenly balanced market sentiment. However, the trend appears bearish below the $1,800 threshold in the short term. Despite some bullish momentum in Ethereum due to strong social and market activity, flagship cryptocurrencies like Bitcoin and Ethereum are signaling bearish trends, impacting the broader market sentiment negatively.

## Summary and Future Trend Prediction:
In conclusion, the current bearish sentiment in the Ethereum market is driven by multiple factors, including corrections near key support levels and uncertainty about future price movements. Traders are advised to proceed with caution and closely monitor market developments. Looking ahead, the future trend for Ethereum remains uncertain, with the possibility of a new low or a potential reversal in the market. It is crucial to stay informed and make well-informed investment decisions in this volatile market environment.